# AgentbayIntegrationToolkit

The AgentbayIntegrationToolkit provides a comprehensive set of tools for interacting with the AgentBay cloud computing platform. It includes tools for file operations, code execution, and command execution within a secure cloud environment.

## Setup

To use the AgentBay integration, you need to get an API key from the [AgentBay Console](https://agentbay.console.aliyun.com/service-management) and configure it as an environment variable:

```bash
export AGENTBAY_API_KEY="your-agentbay-api-key"
export DASHSCOPE_API_KEY="your-dashscope-api-key"
```

### Prerequisites

- **AgentBay account**: Register your AgentBay account.
- **AgentBay API key**: Get your API key from the AgentBay platform dashboard.
- **DashScope API key**: For using Qwen models with the agent. Visit [DashScope Platform](https://bailian.console.aliyun.com/#/home) to get your API key.

### Installation

This toolkit lives in the `langchain-agentbay-integration` package. You also need to install the `agentbay` package to interact with the AgentBay platform:

In [ ]:
%pip install -qU langchain-agentbay-integration wuying-agentbay-sdk

In [ ]:
%pip install -qU langchain-openai langgraph

## Instantiation

Now we can instantiate our toolkit. First, we need to create an AgentBay session:

In [ ]:
import os
import getpass

if not os.environ.get("AGENTBAY_API_KEY"):
    os.environ["AGENTBAY_API_KEY"] = getpass.getpass("AgentBay API key:\n")

from agentbay import AgentBay
from agentbay.session_params import CreateSessionParams

# Create AgentBay session
agent_bay = AgentBay()
params = CreateSessionParams(image_id="code_latest")
result = agent_bay.create(params)
session = result.session

from langchain_agentbay_integration import AgentbayIntegrationToolkit

toolkit = AgentbayIntegrationToolkit(
    session=session
)

## Tools
View available tools:

In [ ]:
tools = toolkit.get_tools()
for tool in tools:
    print(f"Tool: {tool.name}")
    print(f"Description: {tool.description}")
    print()

The toolkit provides four main tools:

1. **WriteFileTool**: Write content to a file in the AgentBay session with either overwrite or append mode.
2. **ReadFileTool**: Read content from a file in the AgentBay session.
3. **RunCodeTool**: Execute Python or JavaScript code in a secure cloud environment with configurable timeout.
4. **ExecuteCommandTool**: Execute shell commands in the AgentBay session with timeout control.

## Use within an agent

In [ ]:
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

# Initialize LLM
llm = ChatOpenAI(
    api_key=os.getenv("DASHSCOPE_API_KEY"),
    base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",
    model=os.getenv("DASHSCOPE_MODEL", "qwen3-max")
)

# Create prompt
prompt = ChatPromptTemplate.from_messages([
    ("system", """You are a helpful assistant with access to AgentBay tools that can write files, read files, execute code, and execute commands.
    
Available tools:
1. write_file - Write content to a file in the AgentBay session. Supports 'overwrite' and 'append' modes.
2. read_file - Read content from a file in the AgentBay session.
3. run_code - Execute code in the AgentBay session. Supported languages are: python, javascript.
4. execute_command - Execute a shell command in the AgentBay session

Use these tools to help the user accomplish their tasks. When using write_file, you can specify the mode parameter to either overwrite (default) or append to a file. When appending content, make sure to include newline characters if needed to separate lines."""),
    ("human", "{input}"),
    ("placeholder", "{agent_scratchpad}")
])

# Create agent
agent = create_tool_calling_agent(llm, toolkit.get_tools(), prompt)
agent_executor = AgentExecutor(agent=agent, tools=toolkit.get_tools(), verbose=True)

In [ ]:
example_query = """Write a Python file '/tmp/script.py' with content 'print("Hello from Python!")\nprint("AgentBay integration successful!")\n' using default mode.
Then run the Python code in that file using the run_code tool.
Next, write a file '/tmp/demo.txt' with content 'First line\n' using default mode.
Then append a second line 'Second line\n' to the same file using append mode.
After that, read the file '/tmp/demo.txt' to verify its content.
Finally, execute command 'cat /tmp/demo.txt' to show the file content."""

result = agent_executor.invoke({"input": example_query})
print(f"Final result: {result['output']}")

## Individual Tool Usage

You can also use tools individually:

In [ ]:
from langchain_agentbay_integration.tools import WriteFileTool, ReadFileTool, RunCodeTool, ExecuteCommandTool

# Create individual tools
write_tool = WriteFileTool(session=session)
read_tool = ReadFileTool(session=session)
code_tool = RunCodeTool(session=session)
command_tool = ExecuteCommandTool(session=session)

# Use tools
write_tool.invoke({"path": "/tmp/test.txt", "content": "Hello World", "mode": "overwrite"})
read_tool.invoke({"path": "/tmp/test.txt"})
code_tool.invoke({"code": "print('Hello from Python!')", "language": "python"})
command_tool.invoke({"command": "ls -la", "timeout_ms": 1000})

## Toolkit Features

The AgentbayIntegrationToolkit provides a comprehensive set of tools for working with the AgentBay platform:

1. **WriteFileTool**: Write content to files in the AgentBay session with support for both overwrite and append modes.
2. **ReadFileTool**: Read content from files in the AgentBay session.
3. **RunCodeTool**: Execute Python or JavaScript code in a secure cloud environment.
4. **ExecuteCommandTool**: Run shell commands with configurable timeout settings.

All tools work within the context of an AgentBay session, which provides a secure and isolated environment for code execution and file operations.

## API reference

API reference documentation will be available in the future.